In [1]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain_ollama.llms import OllamaLLM
from langchain.embeddings import OllamaEmbeddings

from datasets import load_dataset

import cassio
from PyPDF2 import PdfReader

/Users/soumajitbiswas/Desktop/natural-language-processing/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ASTRA_DB_APPLICATION_TOKEN="AstraCS:nmYSfzANwFJqcQhsjbvboAQJ:81029dd121b7e91f15b28438edcf5461380bf225b29d237bd4f4c92e76b23968"
ASTRA_DB_ID="bba9e1b3-c8f2-437c-baf4-e57b207d20fd"

In [3]:
pdfreader=PdfReader('concall-ppt.pdf')

In [4]:
from typing_extensions import Concatenate
raw_text=""
for i,page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [5]:
cassio.init(
    token=ASTRA_DB_APPLICATION_TOKEN,
    database_id=ASTRA_DB_ID
)

In [6]:
llm = OllamaLLM(model="llama3.2")
embedding=OllamaEmbeddings(model="llama3.2")

/var/folders/7j/99g2j2l52sdcr21b1lc2xcjc0000gn/T/ipykernel_40577/1018806059.py:2: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding=OllamaEmbeddings(model="llama3.2")


In [7]:
astra_vector_store = Cassandra(
    embedding=embedding,
    table_name="qa_mini2",
    session=None,
    keyspace=None
)

In [8]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=800,
    chunk_overlap=200,
    length_function=len
)
texts = text_splitter.split_text(raw_text)

In [9]:
texts[:2]

['APL\n/SEC/3 1/2024-25 /11 \n11th November 2024  \nBSE Limited  \nPhiroze Jeejeebhoy Towers,  \nDalal Street,  \nMumbai – 400 001  \nSecurity Code: 500820  National Stock Exchange of India Limited \nExchange Plaza, C -1, Block G,  \nBandra Kurla Complex, Bandra (East),  \nMumbai – 400 051  \nSymbol: ASIANPAINT  \nSir/Madam,  \nS\nub: Intimation under Regulation 30 of the SEBI (Listing Obligations and Disclosure \nRequirements) Regulations, 2015 - Investor Presentation \nP\nursuant to Regulation 30 of the SEBI (Listing Obligations and Disclosure Requirements) \nRegulations, 2015 (“Listing Regulations”), please find enclosed the presentation being made to \nthe investors on the business and financial performance of the Company for the quarter and half  \nyear ended 30th September 2024.  \nThe audi',
 'the investors on the business and financial performance of the Company for the quarter and half  \nyear ended 30th September 2024.  \nThe audi\no recording and transcript of the Investors 

In [10]:
astra_vector_store.add_texts(texts=texts)
astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

In [13]:
query_text = "Any new products introduced?"
answer = astra_vector_index.query(query_text,llm=llm).strip()

In [14]:
answer

'Yes, the company has introduced several new products. Specifically, it mentions:\n\n* New collections of Wallcoverings, Rugs, Furnishings & Fabrics in collaboration with European Designers\n* A new communication on Ultima Protek in the NovPREMIUM CATEGORY\n* Single Component PUD based fleece reinforced liquid applied coating for exposed terrace surfaces\n\nAdditionally, they mention that 2300+ new shades have been introduced, and 264 whites have also been added to their existing range.'